In [4]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import math

In [2]:
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')

In [8]:
df=pd.read_csv('data/jeju_data_ver1/201901-202003.csv')

In [9]:
df.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [ ]:
df['REG_YYMM']=pd.to_datetime(df['REG_YYMM'])

In [ ]:
dfdate=df.set_index('REG_YYMM')
dfdate.head()

In [ ]:
dfdate.resample('M').sum()['AMT'].plot()

In [ ]:
df1=df.set_index(['REG_YYMM','STD_CLSS_NM'])
df1.groupby(level=0)['AMT'].agg({'sum': np.sum})

In [20]:
df1=df.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum})

C:\anaconda\envs\jiwoo\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


In [25]:
df1.head()

AMT
REG_YYMM CARD_SIDO_NM STD_CLSS_NM                  
201901   강원           건강보조식품 소매업          148354746
                      골프장 운영업            1229200312
                      과실 및 채소 소매업        1286979106
                      관광 민예품 및 선물용품 소매업    30656770
                      그외 기타 스포츠시설 운영업      13101500

In [8]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [11]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [12]:
# 날짜 처리
data = pd.read_csv('data/jeju_data_ver1/201901-202003.csv')
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [13]:
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)
df.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3


In [14]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [23]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [24]:
# 훈련
model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(train_features, train_target)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [25]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4,7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [27]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [87]:
temp.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,5.049486e+08
1,202004,강원,골프장 운영업,6.959584e+09
2,202004,강원,과실 및 채소 소매업,1.815465e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,9.745725e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,9.385428e+07


In [29]:
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [84]:
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission.head()
submission['AMT']=[]

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM
id,,,
0,202004,강원,건강보조식품 소매업
1,202004,강원,골프장 운영업
2,202004,강원,과실 및 채소 소매업
3,202004,강원,관광 민예품 및 선물용품 소매업
4,202004,강원,그외 기타 분류안된 오락관련 서비스업


In [50]:
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,5.049486e+08
1,202004,강원,골프장 운영업,6.959584e+09
2,202004,강원,과실 및 채소 소매업,1.815465e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,9.745725e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,9.385428e+07


In [5]:
# 시계열
from fbprophet import Prophet
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

Importing plotly failed. Interactive plots will not work.


In [308]:
def rmse(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    sq_diff = (difference)**2
    
    score = math.sqrt(sq_diff.mean())
    
    return score

### Exponential Smoothing (지수평활법 - Simple, Holt's, Holt-Winter's)
#### 1) Simple Exponential Smoothing
- trend나 seasonality 반영을 하지 못함
- level 정도만 수평선으로 나옴
$$ F_t = F_{t-1} + \alpha(D_{t-1} - F_{t-1}) $$
$$ F_t = (1 - \alpha)F_{t-1} + \alpha D_{t-1} $$
 
- Ft  : 현재 시점의 예측 값
- Ft−1  : 이전 시점의 예측 값
- Dt−1  : 이전 시점의 실제 값
- α  : smoothing 요소, 0 <  α  < 1

![](...%0Aa8UWlm5jdQAAAABJRU5ErkJggg==)

In [309]:
train = df_re.iloc[:13]
test = df_re.iloc[13:]
ses_model = SimpleExpSmoothing(train['AMT'])
ses_result = ses_model.fit()
ses_pred_arr = ses_result.forecast(len(test))
print("RMSE Score of test :",rmse(ses_pred_arr,test['AMT']) )

RMSE Score of test : 1630969483.4739425


C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [323]:
temp=[]
months=[202004,202007]
for month in months:
    for i in df['CARD_SIDO_NM'].unique():
        for j in df['STD_CLSS_NM'].unique():
            temp.append([month, i, j])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])
temp.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM
0,202004,강원,건강보조식품 소매업
1,202004,강원,골프장 운영업
2,202004,강원,과실 및 채소 소매업
3,202004,강원,관광 민예품 및 선물용품 소매업
4,202004,강원,그외 기타 스포츠시설 운영업


In [3]:
ses_pred_list = []
months=[202004,202007]
for month in months:
    for i in df['CARD_SIDO_NM'].unique():
        for j in df['STD_CLSS_NM'].unique():
            df_set=df.loc[(df['CARD_SIDO_NM']==i)&(df['STD_CLSS_NM']==j)]
            df_re=df_set.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum})
            if month==202004:
                try:
                    ses_model=SimpleExpSmoothing(df_re['AMT'])
                    ses_result = ses_model.fit()
                    ses_pred = ses_result.forecast(4)
                    ses_pred_list.append(ses_pred.iloc[0])
                except:
                    ses_pred_list.append(np.sum(df_re['AMT'])/15)
            else:
                try:
                    ses_model=SimpleExpSmoothing(df_re['AMT'])
                    ses_result = ses_model.fit()
                    ses_pred = ses_result.forecast(4)
                    ses_pred_list.append(ses_pred.iloc[3])
                except:
                    ses_pred_list.append(np.sum(df_re['AMT'])/15)

NameError: name 'df' is not defined

In [ ]:
pred = np.array(ses_pred_list)
temp['AMT'] = np.round(pred, 0)
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

## 빈 월수 예외처리 어떻게 하는 게 좋을까?

In [6]:
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)

In [13]:
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.196217e+08
1,202004,강원,골프장 운영업,2.915798e+09
2,202004,강원,과실 및 채소 소매업,1.120324e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,3.065677e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00


In [12]:
for i in df['CARD_SIDO_NM'].unique():
    for j in df['STD_CLSS_NM'].unique():
        df_set=df.loc[(df['CARD_SIDO_NM']==i)&(df['STD_CLSS_NM']==j)]
        df_re=df_set.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum})
        print(df_re)
        try:
            ses_model=SimpleExpSmoothing(df_re['AMT'])
            ses_result = ses_model.fit()
            ses_pred = ses_result.forecast(4)
            submission.loc[(submission['REG_YYMM']==202004)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=ses_pred.iloc[0]
            submission.loc[(submission['REG_YYMM']==202007)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=ses_pred.iloc[3]
        except:
            submission.loc[(submission['REG_YYMM']==202004)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            submission.loc[(submission['REG_YYMM']==202007)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15

C:\anaconda\envs\jiwoo\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  after removing the cwd from sys.path.
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


                                         AMT
REG_YYMM CARD_SIDO_NM STD_CLSS_NM           
201901   강원           건강보조식품 소매업   148354746
201902   강원           건강보조식품 소매업   146715871
201903   강원           건강보조식품 소매업   109439025
201904   강원           건강보조식품 소매업   106548473
201905   강원           건강보조식품 소매업   148115738
201906   강원           건강보조식품 소매업    97960089
201907   강원           건강보조식품 소매업    92505731
201908   강원           건강보조식품 소매업    99023125
201909   강원           건강보조식품 소매업   205353608
201910   강원           건강보조식품 소매업   115917681
201911   강원           건강보조식품 소매업    98084641
201912   강원           건강보조식품 소매업    95232313
202001   강원           건강보조식품 소매업   185008768
202002   강원           건강보조식품 소매업    76505561
202003   강원           건강보조식품 소매업    96059012
                                          AMT
REG_YYMM CARD_SIDO_NM STD_CLSS_NM            
201901   강원           골프장 운영업      1229200312
201902   강원           골프장 운영업      1152240972
201903   강원           골프장 운영업      2760497139
20190

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


                                          AMT
REG_YYMM CARD_SIDO_NM STD_CLSS_NM            
201901   강원           과실 및 채소 소매업  1286979106
201902   강원           과실 및 채소 소매업  1116265883
201903   강원           과실 및 채소 소매업   939227950
201904   강원           과실 및 채소 소매업   827683360
201905   강원           과실 및 채소 소매업  1021472585
201906   강원           과실 및 채소 소매업  1001626059
201907   강원           과실 및 채소 소매업   953346915
201908   강원           과실 및 채소 소매업  1384944417
201909   강원           과실 및 채소 소매업  1313429320
201910   강원           과실 및 채소 소매업  1317877211
201911   강원           과실 및 채소 소매업  1052641622
201912   강원           과실 및 채소 소매업  1021406986
202001   강원           과실 및 채소 소매업  1576556551
202002   강원           과실 및 채소 소매업   867214360
202003   강원           과실 및 채소 소매업   994816943


C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


...

                                            AMT
REG_YYMM CARD_SIDO_NM STD_CLSS_NM              
201901   강원           그외 기타 스포츠시설 운영업  13101500
201902   강원           그외 기타 스포츠시설 운영업   7139500
201903   강원           그외 기타 스포츠시설 운영업   6817720
201904   강원           그외 기타 스포츠시설 운영업   5904100
201905   강원           그외 기타 스포츠시설 운영업   7298000
201906   강원           그외 기타 스포츠시설 운영업   5748000
201907   강원           그외 기타 스포츠시설 운영업   6841100
201908   강원           그외 기타 스포츠시설 운영업   8353400
201909   강원           그외 기타 스포츠시설 운영업   5277600
201910   강원           그외 기타 스포츠시설 운영업   4363900
201911   강원           그외 기타 스포츠시설 운영업   6808100
201912   강원           그외 기타 스포츠시설 운영업   6566300
202001   강원           그외 기타 스포츠시설 운영업   4625300
202002   강원           그외 기타 스포츠시설 운영업   5065100
202003   강원           그외 기타 스포츠시설 운영업   2075000


C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


                                          AMT
REG_YYMM CARD_SIDO_NM STD_CLSS_NM            
201901   강원           그외 기타 종합 소매업  533330540
201902   강원           그외 기타 종합 소매업  516135590
201903   강원           그외 기타 종합 소매업  405132850
201904   강원           그외 기타 종합 소매업  433801770
201905   강원           그외 기타 종합 소매업  440361900
201906   강원           그외 기타 종합 소매업  411086680
201907   강원           그외 기타 종합 소매업  439858230
201908   강원           그외 기타 종합 소매업  460041220
201909   강원           그외 기타 종합 소매업  529805920
201910   강원           그외 기타 종합 소매업  412253560
201911   강원           그외 기타 종합 소매업  419368330
201912   강원           그외 기타 종합 소매업  429629050
202001   강원           그외 기타 종합 소매업  603546720
202002   강원           그외 기타 종합 소매업  436654770
202003   강원           그외 기타 종합 소매업  470200240


C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


KeyboardInterrupt: 

In [249]:
submission.index.name = 'id'
submission.to_csv('submission_ses.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.196217e+08
1,202004,강원,골프장 운영업,2.915798e+09
2,202004,강원,과실 및 채소 소매업,1.120324e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,3.065677e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.180000e+04


### 2)  Holt's Exponential Smoothing
- trend로 데이터를 예측하기 위해 Simple Exponential Smoothing에서 확장한 것이다.
- 예측을 위한 식 외에 level smoothing을 위한 식과 trend smoothing을 위한 식이 포함된다
- 생성된 예측은 선형적으로 나타나기 때문에 예측 범위가 멀어질 수록 over-forecast 되는 경향이 있다.
- Forecast equation :
$$ \hat{y}_{t+h|t} = l_t + hb_t $$
 
- Level equation :

$$ l_{t} = \alpha y_t + (1-\alpha)(l_{t-1} + b_{t-1}) $$
 
- Trend Equation :

$$ b_t = \beta^*(l_t - l_{t-1}) + (1 - \beta^*)b_{t-1} $$
 
- lt  : t 시점에서의 level(수준)의 추정을 나타낸다.

- bt  : t 시점에서의 추세(경사)의 추정을 나타낸다.
- α  : level(수준)에 대한 smoothing parameter이고 다음과 같음 범위를 갖는다. 0 <  α  < 1
- β∗  : trend(추세)에 대한 smoothing parameter이고 다음과 같은 범위를 갖는다. 0 <  β∗  < 1
- level에 대한 식  lt 는 t 시간에 대한 관측치  yt 와 훈련 예측  lt−1+bt−1 의 가중 평균을 나타낸다.
- trend에 대한 식  bt 는  (lt−lt−1) 에 근거한 t시간에 대한 추정치와 이전 추정치인  bt−1 의 가중 평균을 나타낸다.

![](...%0AAElFTkSuQmCC)

In [339]:
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
for i in df['CARD_SIDO_NM'].unique():
    for j in df['STD_CLSS_NM'].unique():
        df_set=df.loc[(df['CARD_SIDO_NM']==i)&(df['STD_CLSS_NM']==j)]
        df_re=df_set.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum})
        try:
            holt_model = Holt(np.array(df_re['AMT']))
            holt_result = holt_model.fit()
            holt_pred = holt_result.forecast(4)
            submission.loc[(submission['REG_YYMM']==202004)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=holt_pred.iloc[0]
            submission.loc[(submission['REG_YYMM']==202007)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=holt_pred.iloc[3]
        except:
            submission.loc[(submission['REG_YYMM']==202004)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            submission.loc[(submission['REG_YYMM']==202007)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15

C:\anaconda\envs\jiwoo\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-pa

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retva

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retva

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retva

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retva

C:\anaconda\envs\jiwoo\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


In [16]:
submission.index.name = 'id'
submission.to_csv('submission_holt.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.196217e+08
1,202004,강원,골프장 운영업,2.915798e+09
2,202004,강원,과실 및 채소 소매업,1.120324e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,3.065677e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00


### 3) Holt-Winter's Exponential Smoothing
- Holt-Winter's 방법은 seasonality를 반영하기 위해 Holt's 방법에서 확장된 것이다.
- Holt-Winter's 방법은 예측식과 3개의 smoothing 식으로 구성되어 있다.
- level에 대한 식  l_t   
- trend에 대한 식  b_t
- seasonal에 대한 요소  s_t
- smoothing parameter에 해당하는  α ,  β∗ ,  γ 
- seasonality의 빈도를 나타내기 위한  m 
- seasonal이 변화하는 형태에 따라 두 가지 방법이 있다.
- additive : seasonal의 변화가 일정하게 지속될 때
- multiplicative : seasonal의 변화가 level에 비례적일 때
- Forecast equation
$$ \hat{y}_{t+h|t} = l_t + hb_t + s_{t + h - m(k+1)} $$
 
- Level equation
$$ l_t = \alpha(y_t - s_{t-m}) + (1 - \alpha)(l_{t-1} + b_{t-1}) $$
 
- Trend equation
$$ b_t = \beta^*(l_t - l_{t-1}) + (1 - \beta^*)b_{t-1} $$
 
- Seasonal equation
$$ s_t = \gamma(y_t - l_{t-1} - b_{t-1}) + (1 - \gamma)s_{t-m} $$

 
- level에 대한 식은 t 시간에 대한 조정된 seasonally 관측치  (y_t−s_t−m) 와 non-seasonally 예측치  (lt−1+bt−1) 의 가중 평균을 나타낸다.
- trend에 대한 식은 Holt's 선형식에서와 동일하다.
- seasonal에 대한 식은 현재 seasonal 지수  (yt−lt−1−bt−1) 와 이전 seasonal 지수( m 기간 전) 사이의 가중 평균을 나타낸다.

![](...)

In [ ]:
%%time
holt_winters_arr = np.array([])
for i in df_month.store_id.unique():
    df_set = df_month[df_month.store_id == i]
    size = int(len(df_set) * 0.7)
    train = df_set[:size]
    test = df_set[size:]
    # 최적값을 찾기 위한 기준
    es_score = 1000000000
    best_period = 0
    # 각 store_id 최적의 seasonal period를 찾기 위한 탐색
    for p in range(2,(len(train)//2) + 1):
        try:
            es_model = ExponentialSmoothing(np.array(train.amount),seasonal_periods=p, trend='add', seasonal='add')
            es_result = es_model.fit()
            es_pred = es_result.forecast(len(test))

            if es_score > mae(test.amount, es_pred):
                es_score = mae(test.amount, es_pred)
                best_period = p
        except:
            continue
    
    # 최적의 parameter을 이용한 3개월 예측과 sum
    set_es_model = ExponentialSmoothing(np.array(df_set.amount), seasonal_periods=best_period, trend='add', seasonal='add')
    set_es_result = set_es_model.fit()
    set_es_pred = set_es_result.forecast(3)
    holt_winters_arr = np.concatenate((holt_winters_arr, np.array([set_es_pred.sum()])))
submission = pd.read_csv('data/jeju_data_ver1/submission.csv', index_col=0)
for i in df['CARD_SIDO_NM'].unique():
    for j in df['STD_CLSS_NM'].unique():
        df_set=df.loc[(df['CARD_SIDO_NM']==i)&(df['STD_CLSS_NM']==j)]
        df_re=df_set.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].agg({'AMT': np.sum})
        train = df_re.iloc[:13]
        test = df_re.iloc[13:]
        # 최적값을 찾기 위한 기준
        es_score = 1000000000
        best_period = 0
        # 각 category 최적의 seasonal period를 찾기 위한 탐색
        for p in range(2,(len(train)//2) + 1):
            try:
                es_model = ExponentialSmoothing(np.array(train['AMT']),seasonal_periods=p, trend='add', seasonal='add')
                es_result = es_model.fit()
                es_pred = es_result.forecast(len(test))
                if es_score > rmse(test['AMT'], es_pred):
                    es_score = rmse(test['AMT'], es_pred)
                    best_period = p
            except:
                continue
        # 최적의 parameter을 이용한 4개월 예측과 sum
        try:
            set_es_model = ExponentialSmoothing(np.array(df_re['AMT']), seasonal_periods=best_period, trend='add', seasonal='add')
            set_es_result = set_es_model.fit()
            set_es_pred = set_es_result.forecast(4)
            submission.loc[(submission['REG_YYMM']==202004)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=set_es_pred.iloc[0]
            submission.loc[(submission['REG_YYMM']==202007)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=set_es_pred.iloc[3]
        except:
            submission.loc[(submission['REG_YYMM']==202004)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
            submission.loc[(submission['REG_YYMM']==202007)&(submission['CARD_SIDO_NM']==i)&(submission['STD_CLSS_NM']==j),'AMT']=np.sum(df_re['AMT'])/15
        
            

## ARIMA(Autoregressive Integrated Moving Average)
- ARIMA 모델은  Yt 을 차분(difference)한 결과로 만들어지 시계열  ∇Yt=Yt−Yt−1 이 ARMA 모형을 따르면 원래의 시계열  Yt 를 ARIMA 모형이라고 한다.
- 만약  d 번 차분한 후예야 시계열  ∇dYt 가 ARMA(p,q) 모형을 따른다면 적분 차수가(order of integration)가 d인 ARIMA 모형으로 ARIMA(p, d, q)로 표기한다.
- q=0인 경우에는 ARI(p,d), q=0인 경우에는 IMA(d,q)로 표기한다.
- p, d, q의 조합을 탐색하며 최적 parameter를 찾고 기준은 fit에 저장되어 있는 AIC(Akaike's Information Criterion)을 기준으로 한다. 다음과 같은 식을 가지며 작을 수록 좋은 모형이다.
- AIC=−2log(Likelihood)+2K
 
- k  : 모델의 추정된 parameter의 갯수
- Likelihood  : 모델의 likelihoood function의 최댓값

In [ ]:
p = list(range(0, 6))
d = [0, 1, 2]
q = list(range(0, 6))

pdq = list(itertools.product(p, d, q))